# Plain Text Tables using Pandas and Real Data
> Sample Covid-19 API Data pulled, flattened, imported into pandas dataframe, and printed to plain text table.

- toc: true
- badges: true
- comments: true
- categories: [general, pandas, covid19, tables, data]

In [45]:
#hide

# WRITING FUNCTIONS

def print_title(string: str, break_before=False, break_after=False) -> str:
    if type(string) != str:
        string = str(string)
    line_length = len(string)
    line_before = '\n' if break_before == True else ''
    line_after = '\n' if break_before == True else ''
    print(f"{line_before}{'-' * line_length}\n{string}\n{'-' * line_length}{line_after}")

## Pull Data into Pandas and Print with Tabulate

### Step 1: Pull Parse Data at a High Level
- `requests` is used to query region 8 data from an IL IDPH endpoint
- `json` parses the chunk of data needed
- In the output below it shows that the json structure currently is broken into an object that lists the date, then has additional nested objects for all of the individual country data for that date.

In [71]:
import requests
import datetime
from pprint import pprint

query_url = 'https://idph.illinois.gov/DPHPublicInformation/api/COVID/GetResurgenceData?regionID=8&daysIncluded=5' # max is 170 days #+ selectedRegion + '&daysIncluded=' + chartRange
page = requests.get(query_url)

# Use just the county level positivity reports
positivity_rates = page.json()['CountyTestPositivityReports']

print_title('2 DAYS OF DATA IN RAW FORM')
pprint(positivity_rates[:2])

--------------------------
2 DAYS OF DATA IN RAW FORM
--------------------------
[{'countyTestPositivities': [{'CountyName': 'DuPage',
                              'dailyPositivity': 0.0,
                              'positive_test': 777,
                              'positivityRollingAvg': 12.3,
                              'regionID': 8,
                              'totalTest': 6411},
                             {'CountyName': 'Kane',
                              'dailyPositivity': 0.0,
                              'positive_test': 517,
                              'positivityRollingAvg': 16.3,
                              'regionID': 8,
                              'totalTest': 3790}],
  'reportDate': '2020-11-24T00:00:00'},
 {'countyTestPositivities': [{'CountyName': 'DuPage',
                              'dailyPositivity': 0.0,
                              'positive_test': 731,
                              'positivityRollingAvg': 12.0,
                              

### Step 2: Flatten Nested Data into Single Level JSON Objects

- It's much easier to import into Pandas when a json object (or any object for that matter, such as a list or tuple) is only one level deep. If data is nested, Pandas' default behavior is to import an entire nested object into a single cell.
- The code below takes the existing list of json objects, and pulls each nested piece of county information out to insert into a new list at the top level.
  - While the code creates the new list, line by line, it also inserts the date (converted to a datetime object) into each section as it's processed.
- In the output below shows a bit of the new json structure. It's a list of non-nested json objects, and each one is associated with date.

In [72]:
# Flatten needed json data into a list so it can be imported to a pandas dataframe
flattened_positivity_rates = []
for dates in positivity_rates:
    for county in dates['countyTestPositivities']:
        county['date'] = datetime.datetime.strptime(dates['reportDate'], "%Y-%m-%dT%H:%M:%S")
        flattened_positivity_rates.append(county)

print_title('SAME 2 DAYS OF DATA, BUT FLATTENED')
pprint(flattened_positivity_rates[:2])

----------------------------------
SAME 2 DAYS OF DATA, BUT FLATTENED
----------------------------------
[{'CountyName': 'DuPage',
  'dailyPositivity': 0.0,
  'date': datetime.datetime(2020, 11, 24, 0, 0),
  'positive_test': 777,
  'positivityRollingAvg': 12.3,
  'regionID': 8,
  'totalTest': 6411},
 {'CountyName': 'Kane',
  'dailyPositivity': 0.0,
  'date': datetime.datetime(2020, 11, 24, 0, 0),
  'positive_test': 517,
  'positivityRollingAvg': 16.3,
  'regionID': 8,
  'totalTest': 3790}]


### Step 3: Import Data into Pandas, a Little Cleanup, and Reorder Columns
- Once each json object in the list is only single level, creating a pandas dataframe out it is a simple one line statement.
- One thing that doesn't come with this dataset is the daily average, but it includes the data needs to calculate it. Again, calculating the data and inserting it as a new column is a single line statement.
- Finally, it's easy to reorder the columns, and leave out unneeded columns in a single statement as well. The output above shows 'dailyPositivity' with a value of 0 - which is the same for every json object in the entire dataset. In the reorder columns statement below removing it is a simple as leaving it out.
- The results can be seen in the output below. This has the data that's needed, but it's still pretty messy.

In [73]:
import numpy as np
import pandas as pd
from tabulate import tabulate

# Create pandas dataframe
region_rates_df = pd.DataFrame(flattened_positivity_rates)

# Calulate daily average based on using total tests and the daily positive tests
region_rates_df['daily_avg'] = (region_rates_df['positive_test'] / region_rates_df['totalTest']) * 100

# Reorder Columns
region_rates_df = pd.DataFrame(region_rates_df, columns=['date', 'regionID', 'CountyName', 'totalTest', 'positive_test', 'daily_avg', 'positivityRollingAvg'])

In [74]:
#hide

region_rates_df

,date,regionID,CountyName,totalTest,positive_test,daily_avg,positivityRollingAvg
0,2020-11-24,8,DuPage,6411,777,12.119794,12.3
1,2020-11-24,8,Kane,3790,517,13.641161,16.3
2,2020-11-25,8,DuPage,6365,731,11.484682,12.0
3,2020-11-25,8,Kane,2847,409,14.365999,15.5
4,2020-11-26,8,DuPage,5301,660,12.450481,11.8
5,2020-11-26,8,Kane,2467,433,17.551682,15.9
6,2020-11-27,8,DuPage,5055,712,14.085064,11.9
7,2020-11-27,8,Kane,2399,413,17.215506,15.4
8,2020-11-28,8,DuPage,4505,474,10.521643,11.7
9,2020-11-28,8,Kane,1568,282,17.984694,15.0


### Step 4: Data Cleanup in Pandas
- Renaming columns is a single step, and the order here doesn't matter (does not change the order in the dataframe)
- Cleaning up the date format is simple, especially because when the json was flattened the date was inserted as a datetime formatted object (which was converted to a pandas "timestamp" by the dataframe import process)
  - Above the date may look fine, but that's because the Jupyter Lab pandas printout removes the hours, minutes, seconds, and microseconds if they are zeros. In reality the actual values looked something like the following: '2020-11-17T00:00:00:00'.
- The original Daily Avg calculation didn't specify decimal place at all. `.round(2)` easy changes that.
- Finally, it makes sense to make certain that the rows are sorted by date.
- In the out put below, everything looks much better, but the back and forth of DuPage and Kane makes it a little difficult to read.

In [75]:
# Rename Columns
region_rates_df.rename(
    columns={'date':'Date', 'regionID':'Region', 'CountyName':'County', 'totalTest':'Total Tests',
             'daily_avg':'Daily Avg', 'positivityRollingAvg':'Rolling Avg', 'positive_test':'Positive'}, 
    inplace=True
)
# Shorten 'Date'
region_rates_df['Date'] = region_rates_df['Date'].dt.strftime('%m/%d/%Y')

# Round Daily Avg to 2 Decimal Places
region_rates_df['Daily Avg'] = region_rates_df['Daily Avg'].round(2)

# Sort dataframe by 'Date'
region_rates_df = region_rates_df.sort_values(['Date'])

In [76]:
#hide

region_rates_df

,Date,Region,County,Total Tests,Positive,Daily Avg,Rolling Avg
0,11/24/2020,8,DuPage,6411,777,12.12,12.3
1,11/24/2020,8,Kane,3790,517,13.64,16.3
2,11/25/2020,8,DuPage,6365,731,11.48,12.0
3,11/25/2020,8,Kane,2847,409,14.37,15.5
4,11/26/2020,8,DuPage,5301,660,12.45,11.8
5,11/26/2020,8,Kane,2467,433,17.55,15.9
6,11/27/2020,8,DuPage,5055,712,14.09,11.9
7,11/27/2020,8,Kane,2399,413,17.22,15.4
8,11/28/2020,8,DuPage,4505,474,10.52,11.7
9,11/28/2020,8,Kane,1568,282,17.98,15.0


Step 5: Print Individual County Date into Plain Text Tables
- The Jupyter Lab printouts of dataframes looks pretty nice, but when printing to a console it looks much more rudementary.
- Below, `.unique()` (actually a 'numpy' library feature) pulls out the county names represented in the dataframe into a 'numpy' array.
  - Similar to objects like lists, the numpy array can be iterated.
- The print statement does several things:
  - Takes the entire dataframe on each pass (may not be a good idea if the dataframe were very large) and filters it to only include rows with the County name represented by the 'name' variable.
  - Sets headers as 'keys', which just specifies that the headers show up. This parameter works differently depending on the kind of input (i.e., if printing a list of lists where index 0 was a list of headers, `headers=list[0]` would be used instead)
  - Sets the table format. Tabulate has a long list of formats available for use.
  - Removes the index (in this case the index that Pandas uses by default)
- The output below shows the results of the work above. Each country is printed in it's own table, with data that is fairly clean and easy to read.

In [77]:
# Create array of unique county names
counties = region_rates_df['County'].unique()

# Print a table for each county
for name in counties:
    # Prints individual country by filtering with a query (i.e., 'County' = name)
    print(tabulate(region_rates_df[(region_rates_df['County'] == name)], headers = 'keys', tablefmt = ('psql'), showindex="never"), '\n')

+------------+----------+----------+---------------+------------+-------------+---------------+
| Date       |   Region | County   |   Total Tests |   Positive |   Daily Avg |   Rolling Avg |
|------------+----------+----------+---------------+------------+-------------+---------------|
| 11/24/2020 |        8 | DuPage   |          6411 |        777 |       12.12 |          12.3 |
| 11/25/2020 |        8 | DuPage   |          6365 |        731 |       11.48 |          12   |
| 11/26/2020 |        8 | DuPage   |          5301 |        660 |       12.45 |          11.8 |
| 11/27/2020 |        8 | DuPage   |          5055 |        712 |       14.09 |          11.9 |
| 11/28/2020 |        8 | DuPage   |          4505 |        474 |       10.52 |          11.7 |
| 11/29/2020 |        8 | DuPage   |          4188 |        431 |       10.29 |          11.4 |
+------------+----------+----------+---------------+------------+-------------+---------------+ 

+------------+----------+----------+--

## Extra: The Code Works for Any Region or Number of Days
- The only hardcoded elements in the previous section were the region number and days in the initial endpoint url.
- This means that any region or number of days can be used and the process will work the same.
- The output below was produced with the same code by requesting 20 days of data from region 7. Region 7 was used because it also only has 2 counties (easier to use in a blog post), but regions with 10 or more counties work fine as well.
- (all of the code above pulled together in one cell can also be viewed by clicking "Expand")

In [82]:
#collapse

import numpy as np
import pandas as pd
from tabulate import tabulate
import requests
import datetime, json, sys

region = 7
days = 20
query_url = f'https://idph.illinois.gov/DPHPublicInformation/api/COVID/GetResurgenceData?regionID={region}&daysIncluded={days}' # max is 170 days #+ selectedRegion + '&daysIncluded=' + chartRange
page = requests.get(query_url)

# Use just the county level positivity reports
try:
    positivity_rates = page.json()['CountyTestPositivityReports']
except json.JSONDecodeError as e:
    print('Selected json object not available. It\'s possible an invalid region was queried')
    raise

# Flatten needed json data into a list so it can be imported to a pandas dataframe
flattened_positivity_rates = []
for dates in positivity_rates:
    for county in dates['countyTestPositivities']:
        county['date'] = datetime.datetime.strptime(dates['reportDate'], "%Y-%m-%dT%H:%M:%S")
        flattened_positivity_rates.append(county)

# Create pandas dataframe
region_rates_df = pd.DataFrame(flattened_positivity_rates)

# Calulate daily average based on using total tests and the daily positive tests
region_rates_df['daily_avg'] = (region_rates_df['positive_test'] / region_rates_df['totalTest']) * 100

# Reorder Columns
region_rates_df = pd.DataFrame(region_rates_df, columns=['date', 'regionID', 'CountyName', 'totalTest', 'positive_test', 'daily_avg', 'positivityRollingAvg'])

# Rename Columns
region_rates_df.rename(
    columns={'date':'Date', 'regionID':'Region', 'CountyName':'County', 'totalTest':'Total Tests',
             'daily_avg':'Daily Avg', 'positivityRollingAvg':'Rolling Avg', 'positive_test':'Positive'}, 
    inplace=True
)
# Shorten 'Date'
region_rates_df['Date'] = region_rates_df['Date'].dt.strftime('%m/%d/%Y')

# Round Daily Avg to 2 Decimal Places
region_rates_df['Daily Avg'] = region_rates_df['Daily Avg'].round(2)

# Sort dataframe by 'Date'
region_rates_df = region_rates_df.sort_values(['Date'])

# Create array of unique county names
counties = region_rates_df['County'].unique()

# Print a table for each county
for name in counties:
    # Prints individual country by filtering with a query (i.e., 'County' = name)
    print(tabulate(region_rates_df[(region_rates_df['County'] == name)], headers = 'keys', tablefmt = ('psql')), '\n')

+----+------------+----------+----------+---------------+------------+-------------+---------------+
|    | Date       |   Region | County   |   Total Tests |   Positive |   Daily Avg |   Rolling Avg |
|----+------------+----------+----------+---------------+------------+-------------+---------------|
|  0 | 11/09/2020 |        7 | Kankakee |           761 |        203 |       26.68 |          19.2 |
|  2 | 11/10/2020 |        7 | Kankakee |          1383 |        220 |       15.91 |          18.8 |
|  4 | 11/11/2020 |        7 | Kankakee |          1450 |        242 |       16.69 |          19.1 |
|  6 | 11/12/2020 |        7 | Kankakee |          1009 |        257 |       25.47 |          20.1 |
|  8 | 11/13/2020 |        7 | Kankakee |          1640 |        353 |       21.52 |          20.3 |
| 10 | 11/14/2020 |        7 | Kankakee |          1143 |        343 |       30.01 |          21.9 |
| 12 | 11/15/2020 |        7 | Kankakee |          1187 |        326 |       27.46 |       